In [12]:
from pydataset import data
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

1. Load the tips dataset.

In [14]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


- Create a column named price_per_person. This should be the total bill divided by the party size.

In [7]:
df['price_per_person'] = df.total_bill / df.size

- Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

**total bill, time, and party size**

- Use select k best to select the top 2 features for predicting tip amount. What are they?

In [20]:
dummies = pd.get_dummies(df[['sex','smoker','day','time']], dummy_na=False, drop_first=[True,True])
df = pd.concat([df,dummies],axis = 1)
df = df.drop(columns=['sex','smoker','day','time'])

In [21]:
train, test = train_test_split(df, random_state=123)

In [22]:
x_train = train.drop(columns='tip')
y_train = train.tip

In [23]:
f_selector = SelectKBest(f_regression, k=2)
f_selector.fit(x_train, y_train)
feature_mask = f_selector.get_support()
f_feature = x_train.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'size']

- Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [26]:
lm = LinearRegression()
rfe = RFE(lm, n_features_to_select=2)
rfe.fit(x_train,y_train)
feature_mask = rfe.support_
rfe_feature = x_train.iloc[:,feature_mask].columns.tolist()

In [27]:
rfe_feature

['size', 'day_Thur']

- Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

**With RFE as the number of features there are to select from changes, so does the relative importance of each of those features. With K best, the test is only run once and the "K" most useful features in that single test are selected.**

2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).